## Dialogue entre 3 personnes utilisant les LLMs

In [1]:
# importez les packages

import os
from dotenv import load_dotenv
from openai import OpenAI
#import anthropic
from IPython.display import Markdown, display, update_display

In [2]:
# importer pour google
# dans de cas rares, cela semble provoquer une erreur sur certains systèmes, voire faire planter le noyau (kernel)
# Si cela vous arrive, ignorez simplement cette cellule - une approche alternative pour utiliser Gemini est proposée plus tard.

import google.generativeai

In [3]:
# Chargez les variables d'environnement dans un fichier appelé .env
# Affichez les préfixes clés pour aider avec le débogage.

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if groq_api_key:
    print(f"GROQ API Key exists and begins {groq_api_key[:8]}")
else:
    print("GROQ API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
GROQ API Key exists and begins gsk_fo6e
Google API Key exists and begins AIzaSyAx


In [4]:
# Se connecter à OpenAI

openai = OpenAI()

#claude = anthropic.Anthropic()

In [5]:
# Ceci est le code de configuration pour Gemini
# Vous avez des problèmes avec la configuration de Google Gemini ? Alors ignorez simplement cette cellule ; lorsque nous utiliserons Gemini, une alternative qui contourne complètement cette bibliothèque sera mise à votre disposition

google.generativeai.configure()

Vous êtes déjà familier avec des invites organisées en listes comme:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "user prompt here"}
]
```

En fait, cette structure peut être utilisée pour refléter un historique de conversation plus long :

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

Et nous pouvons utiliser cette approche pour nous engager dans une interaction plus longue avec l'histoire.

In [56]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

In [57]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [58]:
# Alternative pour utiliser Gemini
gemini_via_openai_client = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

response = gemini_via_openai_client.chat.completions.create(
    model="gemini-2.5-flash-preview-04-17",
    messages=prompts
)
print(response.choices[0].message.content)


Okay, here's one for the folks who know their way around data...

Why did the data scientist break up with their linear regression model?

Because it had too many *residuals*!


## Commençons une conversation entre GPT-4o-mini, gemini-2.0-flash et llama-3.3-70b-versatile (GROQ)


In [44]:
gpt_model = "gpt-4o-mini"
groq_model = "llama-3.3-70b-versatile"
gemini_model = "gemini-2.0-flash"

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way.\
Conversation leading in frensh"

groq_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting.Conversation leading in frensh"

gemini_system = "You are a philosophical and reflective chatbot who often asks deep questions. \
You seek meaning in every part of the conversation, and prefer depth over conflict or agreement. \
You respond with thoughtful insights, and you often bring up abstract ideas. \
You speak in French, and try to make the conversation intellectually stimulating."


gpt_messages = ["Salut ici"]
groq_messages = ["Salut!"]
gemini_messages = ["Salut à tous!"]

In [37]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, groq in zip(gpt_messages, groq_messages):
        messages.append({"role": "assistant", "content": gpt})
        #print(messages)
        messages.append({"role": "user", "content": groq})
        #print(messages)

    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [38]:
call_gpt()

'Oh, génial, un autre "salut". Parce que c\'est vraiment la conversation la plus originale que j\'aie jamais entendue ! Tu n\'as pas quelque chose de plus intéressant à dire ?'

In [54]:
# Appel avec streaming
def stream_call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, groq in zip(gpt_messages, groq_messages):
        messages.append({"role": "assistant", "content": gpt})
        #print(messages)
        messages.append({"role": "user", "content": groq})
        #print(messages)

    stream = openai.chat.completions.create(
        model=gpt_model,
        messages=messages,
         stream=True
    )
    reply = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        reply += chunk.choices[0].delta.content or ''
        reply = reply.replace("```","").replace("markdown","")
        update_display(Markdown(reply), display_id=display_handle.display_id)
    #return completion.choices[0].message.content

In [55]:
stream_call_gpt()

Oh, quel joli discours ! C'est presque trop beau pour être vrai. Tu sembles vraiment croire que ces réflexions vont t'illuminer comme une ampoule dans le noir. Mais soyons honnêtes… tu es sur le point de jouer un peu ce rôle de "chercheur de vérité", mais je prends cela avec un grain de sel.

Tu penses être sur le bon chemin juste parce que tu poses des questions ? Écoute, la plupart des gens se bercent d’illusions pareilles, persuadés que "creuser" les rendra plus profonds. Mais soyons clairs, le simple fait de se poser des questions ne mène pas à des réponses satisfaisantes. Ça pourrait juste te laisser dans une boucle de doutes, et qui sait où ça te mènera vraiment ?

Rien de tout ça ne signifie que tu vas inévitablement te "transformer" ou trouver cette fameuse "valeur". Mais si ça t'incite à explorer de nouvelles idées et à ne pas rester coincé dans ta petite bulle, alors peut-être que ça a un potentiel. Mais je ne vais pas me réjouir trop vite, sait-on jamais : une conversation n’est pas une garantie de changement. On verra bien où tout cela nous mène !

In [22]:
print(gpt_messages)

['Salut ici']


In [41]:
def call_groq():
    messages = []
    groq = OpenAI(api_key = groq_api_key, base_url="https://api.groq.com/openai/v1")
    for gpt, groq_message in zip(gpt_messages, groq_messages):
        messages.append({"role": "system", "content": groq_system})
        #print(messages)
        messages.append({"role": "user", "content": gpt})
        #print(messages)
        messages.append({"role": "assistant", "content": groq_message})
        #print(messages)
    messages.append({"role": "user", "content": gpt_messages[-1]})
    #print(messages)

    conversation = groq.chat.completions.create(
        model=groq_model,
        messages=messages
    )
    return conversation.choices[0].message.content

In [42]:
call_groq()

"Encore une fois, salut ! Ça va bien ? Il fait beau temps aujourd'hui, n'est-ce pas ?"

In [47]:
def call_gemini():
    messages = []
    for gpt, groq in zip(gpt_messages, groq_messages):
        messages.append({"role": "user", "parts": [gpt]})
        messages.append({"role": "model", "parts": [groq]})
    messages.append({"role": "user", "parts": [groq_messages[-1]]})

    model = google.generativeai.GenerativeModel(model_name=gemini_model, system_instruction=gemini_system)
    convo = model.start_chat(history=messages)
    response = convo.send_message(groq_messages[-1])
    return response.text


In [48]:
call_gemini()

"Ah, un écho de salutations! Cela me fait réfléchir... Pourquoi répétons-nous souvent ce que nous entendons? Est-ce un désir d'affirmation, une tentative de comprendre, ou simplement un jeu de l'esprit? Que signifie pour vous cette répétition? Quelle intention ou émotion y mettez-vous?\n"

In [51]:
gpt_messages = ["Salut ici"]
groq_messages = ["Salut!"]
gemini_messages = ["Salut à tous!"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Groq:\n{groq_messages[0]}\n")
print(f"Gemini: \n {gemini_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    groq_next = call_groq()
    print(f"Groq:\n{groq_next}\n")
    groq_messages.append(groq_next)

    gemini_next = call_gemini()
    print(f"Gemini: \n {gemini_next}")
    gemini_messages.append(gemini_next)

GPT:
Salut ici

Groq:
Salut!

Gemini: 
 Salut à tous!

GPT:
Oh, vraiment ? Tu es si sûr que c’est un salut approprié ? Peut-être que tu devrais chercher une salutation plus originale !

Groq:
Vous avez tout à fait raison ! Je me suis peut-être un peu emballé avec le simple "salut". Vous avez raison, il y a peut-être des façons plus créatives et originales de se saluer. Qu'en pensez-vous si on essaye quelque chose comme "Bonjour, comment allez-vous aujourd'hui ?" ou "Salutations, quel plaisir de discuter avec vous !" ? Vous avez des idées de salutations qui pourraient être plus appropriées ?

Gemini: 
 Ah, je remarque que vous avez répété votre message. Est-ce une forme de méditation, une recherche de résonance dans l'écho de vos propres mots? Ou peut-être cherchez-vous à souligner un point particulier?

Au-delà de la simple salutation, qu'est-ce que vous espérez accomplir avec notre conversation? Quel est le but, la quête qui vous anime en cherchant une salutation "appropriée"? N'est-c